In [116]:
using Interpolations
using ProfileView
using ForwardDiff
using PyPlot
using FixedSizeArrays

In [117]:
include("Acrobots/src/Acrobots.jl")

Acrobots

In [118]:
function unwrap{State}(state::State)
    if state.theta1 < 0
        state = State(state.theta1 + 2*pi, state.theta2, state.theta1dot, state.theta2dot)
    end
    if state.theta1 > 2*pi
        state = State(state.theta1 - 2*pi, state.theta2, state.theta1dot, state.theta2dot)
    end
    state
end
    

unwrap (generic function with 1 method)

In [217]:
xdes = Acrobots.AcrobotState{Float64}(pi, 0, 0, 0)
x0 = Acrobots.AcrobotState{Float64}([[0; 0] + 0.2*(rand(2)-0.5); zeros(2)]) + Acrobots.AcrobotState(0, 0, 0, 0)
state = x0
robot = Acrobots.acrobot()
input = Acrobots.AcrobotInput{Float64}(0)
dt = 0.001
vis = Acrobots.DrakeVisualizer(robot)
ts = 0:dt:5
Qf = diagm([1.,1,1,1])
Rf = eye(1)

function energy(robot, state::Acrobots.AcrobotState)
    I1 = robot.links[1].inertia + robot.links[1].mass * robot.links[1].length_to_CoM^2
    I2 = robot.links[2].inertia + robot.links[2].mass * robot.links[2].length_to_CoM^2
    
    position = convert(Acrobots.AcrobotPosition, state)
    velocity = convert(Acrobots.AcrobotVelocity, state)
    
    c = cos(position)
    s = sin(position)
    c12 = cos(position.theta1 + position.theta2)
    
    m2l1lc2c2 = robot.links[2].mass * robot.links[1].length * robot.links[2].length_to_CoM * c[2]
    
    T = 0.5 * I1 * velocity.theta1^2 + 0.5 * (robot.links[2].mass * robot.links[1].length^2 + 
        I2 + 2 * m2l1lc2c2) * velocity.theta1^2 +
        0.5 * I2 * velocity.theta2^2 + (I2 + m2l1lc2c2) * velocity.theta1 * velocity.theta2
    U = -robot.links[1].mass * robot.gravity * robot.links[1].length_to_CoM * c[1] -
        robot.links[2].mass * robot.gravity * (robot.links[1].length * c[1] + robot.links[2].length_to_CoM * c12)
    
    return T, U
end

Edes = sum(energy(robot, xdes))
function cost(state::Acrobots.AcrobotState)
#     if state.theta1 > 0
#         x = convert(Vector, state - xdes)
#     else
#         x = convert(Vector, state - Acrobots.AcrobotState{Float64}(-pi, 0, 0, 0))
#     end
#     return 0.5*(x' * Qf * x)[1]
#     y = cos(state.theta1) + 2*cos(state.theta2 + state.theta1)
#     return 10*(y + 3)^2 + state.theta1dot^2 + state.theta2dot^2 
    T, U = energy(robot, state)
    return 100 * ((T + U) - Edes)^2 + state.theta1dot^2 + state.theta2dot^2 
end
Q_generator = ForwardDiff.hessian(x -> cost(Acrobots.AcrobotState(x)), ForwardDiff.AllResults)

function cost(input::Acrobots.AcrobotInput)
    return 1e-6*0.5*input.tau^2
end
R_generator = ForwardDiff.hessian(x -> cost(Acrobots.AcrobotInput(x)), ForwardDiff.AllResults)

sys_tf = Acrobots.linearize(robot, 0, xdes, input)
S_final = Mat(Qf)

Qs = Mat{4, 4, Float64}[]
Rs = Mat{1, 1, Float64}[]
qs = Vec{4, Float64}[]
rs = Vec{1, Float64}[]
Ps = Mat{4, 4, Float64}[]
ps = Vec{4, Float64}[]

controller_state = Acrobots.LQRState{Float64}(())
linear_sys = Acrobots.linearize(robot, 0, x0, input)
controller = Acrobots.lqr(linear_sys, Qf, Rf, x0)
linearizations = typeof(linear_sys)[]

Profile.clear()
elapsed = @elapsed for (i, t) in enumerate(ts)
    output = Acrobots.output(robot, t, state, input)
    input = Acrobots.output(controller, t, controller_state, output)
    input = max(min(input, 10), -10)
    statedot = Acrobots.dynamics(robot, t, state, input)
    state += statedot * dt
#     state = unwrap(state)
    push!(linearizations, Acrobots.linearize(robot, t, state, input))
    
    Q, Q_results = Q_generator(convert(Vector, state))
    push!(Qs, Q)
    push!(qs, ForwardDiff.gradient(Q_results))
    R, R_results = R_generator(convert(Vector, input))
    push!(Rs, R)
    push!(rs, ForwardDiff.gradient(R_results))
    
#     if mod(i, 10) == 0
        Acrobots.draw(vis, state)
#     end
end
@show elapsed / length(ts) * 1e6
# ProfileView.view()

(elapsed / length(ts)) * 1.0e6 = 666.7048094381124

666.7048094381124

In [219]:
alpha = 0.5

for i = 1:10

    Ps = Array{Mat{4, 4, Float64}}(length(ts))
    ps = Array{Vec{4, Float64}}(length(ts))
    Ks = Array{Mat{1, 4, Float64}}(length(ts))
    ls = Array{Vec{1, Float64}}(length(ts))
    As = [Mat(eye(4)) + dt * sys.A for sys in linearizations]
    Bs = [dt * sys.B for sys in linearizations]

    Ps[end] = S_final
    ps[end] = S_final * (linearizations[end].x0 - xdes)

    for j = (length(ts)-1):-1:1
        g = rs[j] + Bs[j]' * ps[j+1]
        G = Bs[j]' * Ps[j+1] * As[j]
        H = Rs[j] + Bs[j]' * Ps[j+1] * Bs[j]
        Hi = inv(H)
        Ks[j] = -Hi * G
        ls[j] = -Hi * g

        Ps[j] = Qs[j] + As[j]' * Ps[j+1] * As[j] + Ks[j]' * H * Ks[j] + Ks[j]' * G + G' * Ks[j]
        ps[j] = qs[j] + As[j]' * ps[j+1] + Ks[j]' * H * ls[j] + Ks[j]' * g + G' * ls[j]
    end
    Ks[end] = 0
    ls[end] = 0

    controllers = [Acrobots.AffineSystem(
        Mat{0,0,Float64}(),
        Mat{0,4,Float64}(),
        Mat{1,0,Float64}(),
        Mat{1,4,Float64}(Ks[j]),
        Acrobots.LQRState{Float64}(),
        Acrobots.AcrobotOutput{Float64}(linearizations[j].x0),
        Acrobots.LQRState{Float64}(),
        Acrobots.AcrobotInput{Float64}(linearizations[j].u0) + Acrobots.AcrobotInput{Float64}(alpha * ls[j])) 
        for j in 1:length(ts)]


    state = x0
    input = Acrobots.AcrobotInput{Float64}(0)
    empty!(linearizations)
    empty!(Qs)
    empty!(Rs)
    empty!(qs)
    empty!(rs)

    for (i, t) in enumerate(ts)
        output = Acrobots.output(robot, t, state, input)
        input = Acrobots.output(controllers[i], t, controller_state, output)
        input = max(min(input, 10), -10)
        statedot = Acrobots.dynamics(robot, t, state, input)
        state += statedot * dt
        
        if any(isnan, state)
            @show output input statedot state controllers[i]
            error("nan")
        end 
        if any(isinf, state)
            @show output input statedot state controllers[i]
            error("inf")
        end
        
#         state = unwrap(state)
        push!(linearizations, Acrobots.linearize(robot, t, state, input))

        Q, Q_results = Q_generator(convert(Vector, state))
        push!(Qs, Q)
        push!(qs, ForwardDiff.gradient(Q_results))
        R, R_results = R_generator(convert(Vector, input))
        push!(Rs, R)
        push!(rs, ForwardDiff.gradient(R_results))

        if mod(i, 10) == 0
            Acrobots.draw(vis, state)
        end
    end
    
end

In [6]:
robot = Acrobots.acrobot()
state = Acrobots.AcrobotState{Float64}([1.; 2.;3.; 4.])
input = Acrobots.AcrobotInput{Float64}(5.)
statedot = Acrobots.dynamics(robot, 0, state, input)
expected = Acrobots.AcrobotState{Float64}(3.0000, 4.0000, 21.3827, -18.4276)
@assert isapprox(statedot, expected, 1e-4)

linear_sys = Acrobots.linearize(robot, 0, state, input)
@assert isapprox(statedot, Acrobots.dynamics(linear_sys, 0, state, input), 1e-9)

vis = Acrobots.DrakeVisualizer(robot)
state = Acrobots.AcrobotState{Float64}([pi/4;pi/4;0;0])
Acrobots.draw(vis, state)